In [1]:
import numpy as np
import pyaudio as pa
import os, time
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline

import nemo
import nemo.collections.asr as nemo_asr

[NeMo W 2021-03-25 03:25:28 experimental:28] Module <class 'nemo.collections.asr.models.clustering_diarizer.ClusteringDiarizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-03-25 03:25:28 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
# sample rate, Hz
SAMPLE_RATE = 16000

In [3]:
#mbn_model = nemo_asr.models.EncDecClassificationModel.from_pretrained("MatchboxNet-3x1x64-v2")
mbn_model = nemo_asr.models.EncDecClassificationModel.from_pretrained("MatchboxNet-3x1x64-v2-subset-task")

[NeMo I 2021-03-25 03:26:14 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/MatchboxNet-3x1x64-v2-subset-task.nemo to /home/cjbayron/.cache/torch/NeMo/NeMo_1.0.0rc2/MatchboxNet-3x1x64-v2-subset-task/da747cb40572af6f3774d7e33841815c/MatchboxNet-3x1x64-v2-subset-task.nemo
[NeMo I 2021-03-25 03:26:17 common:615] Instantiating model from pre-trained checkpoint


[NeMo W 2021-03-25 03:26:17 modelPT:133] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /media/smajumdar/data/Datasets/GSC_subset/google_speech_recognition_v2/train_manifest.json
    sample_rate: 16000
    labels:
    - 'yes'
    - 'no'
    - up
    - down
    - left
    - right
    - 'on'
    - 'off'
    - stop
    - go
    - unknown
    - silence
    batch_size: 128
    shuffle: true
    augmentor:
      shift:
        prob: 1.0
        min_shift_ms: -5.0
        max_shift_ms: 5.0
      white_noise:
        prob: 1.0
        min_level: -90
        max_level: -46
    num_workers: 12
    
[NeMo W 2021-03-25 03:26:17 modelPT:140] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /media/sma

[NeMo I 2021-03-25 03:26:17 modelPT:376] Model EncDecClassificationModel was successfully restored from /home/cjbayron/.cache/torch/NeMo/NeMo_1.0.0rc2/MatchboxNet-3x1x64-v2-subset-task/da747cb40572af6f3774d7e33841815c/MatchboxNet-3x1x64-v2-subset-task.nemo.


In [4]:
!du -h /home/cjbayron/.cache/torch/NeMo/NeMo_1.0.0rc2/MatchboxNet-3x1x64-v2-subset-task/da747cb40572af6f3774d7e33841815c/*

304K	/home/cjbayron/.cache/torch/NeMo/NeMo_1.0.0rc2/MatchboxNet-3x1x64-v2-subset-task/da747cb40572af6f3774d7e33841815c/MatchboxNet-3x1x64-v2-subset-task.nemo


In [10]:
mbn_model

EncDecClassificationModel(
  (spec_augmentation): SpectrogramAugmentation(
    (spec_cutout): SpecCutout()
    (spec_augment): SpecAugment()
  )
  (crop_or_pad): CropOrPadSpectrogramAugmentation()
  (preprocessor): AudioToMFCCPreprocessor(
    (featurizer): MFCC(
      (amplitude_to_DB): AmplitudeToDB()
      (MelSpectrogram): MelSpectrogram(
        (spectrogram): Spectrogram()
        (mel_scale): MelScale()
      )
    )
  )
  (encoder): ConvASREncoder(
    (encoder): Sequential(
      (0): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(64, 64, kernel_size=[11], stride=[1], padding=(5,), dilation=[1], groups=64, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(64, 128, kernel_size=(1,), stride=(1,), bias=False)
          )
          (2): BatchNorm1d(128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        )
        (mout): Sequential(
          (0): ReLU()
          (1): Dropout(p=0

In [16]:
for param in mbn_model.decoder.parameters():
    print(param)

Parameter containing:
tensor([[-0.0271, -0.3415, -0.2635,  ...,  0.4301, -0.0387, -0.3152],
        [-0.0592,  0.1672,  0.8944,  ...,  0.4101, -0.0634, -0.2409],
        [-0.0984, -0.1541, -0.1755,  ..., -0.1018,  0.4224, -0.3147],
        ...,
        [ 0.1432,  0.6111, -0.4245,  ..., -0.1463, -0.3457,  0.0450],
        [ 0.3348,  0.7647,  0.1555,  ...,  0.3096, -0.4508,  0.4696],
        [-0.0530, -0.1530, -0.0728,  ..., -0.1274, -0.2428, -0.1959]],
       requires_grad=True)
Parameter containing:
tensor([-0.6165, -0.3133,  0.9215,  0.0745,  0.3984, -0.2405,  0.0378,  0.3498,
         0.0644,  0.0400,  0.1372, -0.8501], requires_grad=True)


In [34]:
# freeze all params
for param in mbn_model.parameters():
    param.requires_grad = False

# unfreeze decoder
for param in mbn_model.decoder.parameters():
    param.requires_grad = True

In [37]:
# check layers to train
for param in mbn_model.named_parameters():
    if param[1].requires_grad:
        print(param[0])

decoder.decoder_layers.0.weight
decoder.decoder_layers.0.bias


In [ ]:
# check: https://towardsdatascience.com/train-conversational-ai-in-3-lines-of-code-with-nemo-and-lightning-a6088988ae37